In [ ]:
%matplotlib notebook

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
#from config import user_name, user_pass
 
from config import conn_str  
from dboperations import connect2db, insertvalues

In [5]:
#Import Streaming database from CSV
streaming_db = pd.read_csv('../Resources/MoviesOnStreamingPlatforms.csv')
streaming_db.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [6]:
#Drop column "Unamed: 0" and Type
del streaming_db["Unnamed: 0"]
del streaming_db['Type']

In [7]:
#Drop rows where ratings are not available
streaming_df = streaming_db.dropna(subset=['IMDb', 'Rotten Tomatoes'], how='any')
streaming_df.head(5)

,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Directors,Genres,Country,Language,Runtime
0,1,Inception,2010,13+,8.8,87%,1,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [8]:
#Split Directors column into first and second director columns
streaming_df[['First Director', 'Second_Director']] = streaming_df.Directors.str.split(",", n=1, expand=True)

/Users/znurtokatli/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [9]:
#Split Directors column into first and second director columns
streaming_df[['Second Director', 'Third Director']] = streaming_df.Second_Director.str.split(",", n=1, expand=True)

In [10]:
#Delete duplicate columns or those not required
del streaming_df['Directors']
del streaming_df['Second_Director']
del streaming_df['Third Director']

streaming_df.head()

,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Genres,Country,Language,Runtime,First Director,Second Director
0,1,Inception,2010,13+,8.8,87%,1,0,0,0,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0,Christopher Nolan,None
1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,"Action,Sci-Fi",United States,English,136.0,Lana Wachowski,Lilly Wachowski
2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,"Action,Adventure,Sci-Fi",United States,English,149.0,Anthony Russo,Joe Russo
3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,"Adventure,Comedy,Sci-Fi",United States,English,116.0,Robert Zemeckis,None
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,Western,"Italy,Spain,West Germany",Italian,161.0,Sergio Leone,None


In [11]:
#Clean data to reflect primary and secondary filming locations and place and populate corresponding new columns
streaming_df[['Primary Location', 'Secondary_Location']] = streaming_df.Country.str.split(",", n=1, expand=True)


/Users/znurtokatli/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [12]:
#Perform a split between second and third filming locations  
streaming_df[['Secondary Location', 'Third Location']] = streaming_df.Secondary_Location.str.split(",", n=1, expand=True)

In [13]:
#Drop Secondary_Location & Third Location
del streaming_df['Secondary_Location']
del streaming_df['Third Location']
del streaming_df['Country']
streaming_df.head(5)

,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Genres,Language,Runtime,First Director,Second Director,Primary Location,Secondary Location
0,1,Inception,2010,13+,8.8,87%,1,0,0,0,"Action,Adventure,Sci-Fi,Thriller","English,Japanese,French",148.0,Christopher Nolan,None,United States,United Kingdom
1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,"Action,Sci-Fi",English,136.0,Lana Wachowski,Lilly Wachowski,United States,None
2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,"Action,Adventure,Sci-Fi",English,149.0,Anthony Russo,Joe Russo,United States,None
3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,"Adventure,Comedy,Sci-Fi",English,116.0,Robert Zemeckis,None,United States,None
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,Western,Italian,161.0,Sergio Leone,None,Italy,Spain


In [14]:
#Split Language column into primary and secondary languages and populate corresponding new columns
streaming_df[['Primary Language', 'Secondary_Language']] = streaming_df.Language.str.split(",", n=1, expand=True)


/Users/znurtokatli/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [15]:
#Split Language column into primary and secondary languages and populate corresponding new columns
streaming_df[['Secondary Language', 'Third Language']] = streaming_df.Secondary_Language.str.split(",", n=1, expand=True)


In [16]:
#Drop Language and Secondary_language
del streaming_df['Language']
del streaming_df['Secondary_Language']
del streaming_df['Third Language']
streaming_df.head()

,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Genres,Runtime,First Director,Second Director,Primary Location,Secondary Location,Primary Language,Secondary Language
0,1,Inception,2010,13+,8.8,87%,1,0,0,0,"Action,Adventure,Sci-Fi,Thriller",148.0,Christopher Nolan,None,United States,United Kingdom,English,Japanese
1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,"Action,Sci-Fi",136.0,Lana Wachowski,Lilly Wachowski,United States,None,English,None
2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,"Action,Adventure,Sci-Fi",149.0,Anthony Russo,Joe Russo,United States,None,English,None
3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,"Adventure,Comedy,Sci-Fi",116.0,Robert Zemeckis,None,United States,None,English,None
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,Western,161.0,Sergio Leone,None,Italy,Spain,Italian,None


In [17]:
#Reorder columns for easier analysis
ordered_df = streaming_df[['ID','Title','Year','Age','IMDb', 'Rotten Tomatoes', 'Netflix', 'Hulu', 'Prime Video', 'Disney+',\
                        'First Director', 'Second Director', 'Primary Location', 'Secondary Location', 'Primary Language',\
                        'Secondary Language','Runtime','Genres']]
ordered_df.head()

,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,First Director,Second Director,Primary Location,Secondary Location,Primary Language,Secondary Language,Runtime,Genres
0,1,Inception,2010,13+,8.8,87%,1,0,0,0,Christopher Nolan,None,United States,United Kingdom,English,Japanese,148.0,"Action,Adventure,Sci-Fi,Thriller"
1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,Lana Wachowski,Lilly Wachowski,United States,None,English,None,136.0,"Action,Sci-Fi"
2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,Anthony Russo,Joe Russo,United States,None,English,None,149.0,"Action,Adventure,Sci-Fi"
3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,Robert Zemeckis,None,United States,None,English,None,116.0,"Adventure,Comedy,Sci-Fi"
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,Sergio Leone,None,Italy,Spain,Italian,None,161.0,Western


In [18]:
#split the genres into seperate genre columns
ordered_df[['First Genre', 'Second Genre', 'Third Genre', 'Fourth Genre', 'Fifth Genre', 'Sixth Genre', 'Seventh Genre', 'Eigth Genre']] = ordered_df.Genres.str.split(",", expand=True)
ordered_df

/Users/znurtokatli/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,...,Runtime,Genres,First Genre,Second Genre,Third Genre,Fourth Genre,Fifth Genre,Sixth Genre,Seventh Genre,Eigth Genre
0,1,Inception,2010,13+,8.8,87%,1,0,0,0,...,148.0,"Action,Adventure,Sci-Fi,Thriller",Action,Adventure,Sci-Fi,Thriller,None,None,None,None
1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,...,136.0,"Action,Sci-Fi",Action,Sci-Fi,None,None,None,None,None,None
2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,...,149.0,"Action,Adventure,Sci-Fi",Action,Adventure,Sci-Fi,None,None,None,None,None
3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,...,116.0,"Adventure,Comedy,Sci-Fi",Adventure,Comedy,Sci-Fi,None,None,None,None,None
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,...,161.0,Western,Western,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16671,16672,George of the Jungle 2,2003,7+,3.3,17%,0,0,0,1,...,87.0,"Adventure,Comedy,Family",Adventure,Comedy,Family,None,None,None,None,None
16677,16678,That Darn Cat,1997,7+,4.7,13%,0,0,0,1,...,116.0,"Comedy,Crime,Family,Thriller",Comedy,Crime,Family,Thriller,None,None,None,None
16687,16688,Kazaam,1996,7+,3.0,6%,0,0,0,1,...,93.0,"Comedy,Family,Fantasy,Musical",Comedy,Family,Fantasy,Musical,None,None,None,None
16705,16706,Meet the Deedles,1998,7+,4.1,7%,0,0,0,1,...,93.0,"Comedy,Family",Comedy,Family,None,None,None,None,None,None


In [19]:
#Delete Genres column and confirm order of remaining columns
del ordered_df['Genres']
ordered_df.columns

Index(['ID', 'Title', 'Year', 'Age', 'IMDb', 'Rotten Tomatoes', 'Netflix',
       'Hulu', 'Prime Video', 'Disney+', 'First Director', 'Second Director',
       'Primary Location', 'Secondary Location', 'Primary Language',
       'Secondary Language', 'Runtime', 'First Genre', 'Second Genre',
       'Third Genre', 'Fourth Genre', 'Fifth Genre', 'Sixth Genre',
       'Seventh Genre', 'Eigth Genre'],
      dtype='object')

In [20]:
streams_df = ordered_df.astype({"Netflix":bool, "Hulu":bool, "Prime Video":bool, "Disney+":bool})
streams_df

,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,...,Secondary Language,Runtime,First Genre,Second Genre,Third Genre,Fourth Genre,Fifth Genre,Sixth Genre,Seventh Genre,Eigth Genre
0,1,Inception,2010,13+,8.8,87%,True,False,False,False,...,Japanese,148.0,Action,Adventure,Sci-Fi,Thriller,None,None,None,None
1,2,The Matrix,1999,18+,8.7,87%,True,False,False,False,...,None,136.0,Action,Sci-Fi,None,None,None,None,None,None
2,3,Avengers: Infinity War,2018,13+,8.5,84%,True,False,False,False,...,None,149.0,Action,Adventure,Sci-Fi,None,None,None,None,None
3,4,Back to the Future,1985,7+,8.5,96%,True,False,False,False,...,None,116.0,Adventure,Comedy,Sci-Fi,None,None,None,None,None
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,True,False,True,False,...,None,161.0,Western,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16671,16672,George of the Jungle 2,2003,7+,3.3,17%,False,False,False,True,...,None,87.0,Adventure,Comedy,Family,None,None,None,None,None
16677,16678,That Darn Cat,1997,7+,4.7,13%,False,False,False,True,...,French,116.0,Comedy,Crime,Family,Thriller,None,None,None,None
16687,16688,Kazaam,1996,7+,3.0,6%,False,False,False,True,...,None,93.0,Comedy,Family,Fantasy,Musical,None,None,None,None
16705,16706,Meet the Deedles,1998,7+,4.1,7%,False,False,False,True,...,None,93.0,Comedy,Family,None,None,None,None,None,None


In [21]:
#output to CSV
streams_df.to_csv("../Output/streams_df.csv", index=False, header=True)

Convert back to Database

In [ ]:
connect2db(conn_str)
insertvalues()

In [ ]:
#Connect to Local Database
#rds_connection_string = (f'{user_name}:{user_pass}@)localhost:5432/streaming_db')
#engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
#Use pandas to load csv converted DataFrame into database
#streams_df.to_sql(name='streaming_db', con=engine, if_exists='append', index=False)